<a href="https://colab.research.google.com/github/thatsmygirl/AI-project/blob/main/%EC%A0%84%EC%9D%B4%ED%95%99%EC%8A%B5.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

In [1]:
import tensorflow as tf
from tensorflow import keras

import matplotlib.pyplot as plt
import numpy as np

import os

In [26]:
image_size = 224
target_size = (image_size, image_size)
input_shape = (image_size, image_size, 3)

batch_size = 32
epochs = 20

In [41]:
base_dir = "../content/drive/MyDrive/dataset/New Plant Diseases Dataset(Augmented)/New Plant Diseases Dataset(Augmented)"
train_dir = os.path.join(base_dir,"train")
test_dir = os.path.join(base_dir,"valid")
!rmdir /content/drive/My Drive/DeepCID/model_cnn/.ipynb_checkpoints

rmdir: failed to remove '/content/drive/My': No such file or directory
rmdir: failed to remove 'Drive/DeepCID/model_cnn/.ipynb_checkpoints': No such file or directory


In [42]:
train_datagen = keras.preprocessing.image.ImageDataGenerator(rescale = 1/255.0,
                                                             shear_range = 0.2,
                                                             zoom_range = 0.2,
                                                             width_shift_range = 0.2,
                                                             height_shift_range = 0.2,
                                                             fill_mode="nearest")

test_datagen = keras.preprocessing.image.ImageDataGenerator(rescale = 1/255.0)

In [43]:
train_data = train_datagen.flow_from_directory(train_dir,
                                               target_size = (image_size, image_size),
                                               batch_size = batch_size,
                                               class_mode = "categorical")

test_data = test_datagen.flow_from_directory(test_dir,
                                             target_size = (image_size, image_size),
                                             batch_size = batch_size,
                                             class_mode = "categorical")

Found 18504 images belonging to 10 classes.
Found 4626 images belonging to 10 classes.


In [44]:
categories = list(train_data.class_indices.keys())
print(train_data.class_indices)

{'Apple___Apple_scab': 0, 'Apple___Black_rot': 1, 'Apple___Cedar_apple_rust': 2, 'Apple___healthy': 3, 'Blueberry___healthy': 4, 'Cherry_(including_sour)___Powdery_mildew': 5, 'Cherry_(including_sour)___healthy': 6, 'Corn_(maize)___Cercospora_leaf_spot Gray_leaf_spot': 7, 'Corn_(maize)___Common_rust_': 8, 'Corn_(maize)___healthy': 9}


In [31]:
import json
with open('class_indices.json','w') as f:
  json.dump(train_data.class_indices, f)

from IPython.display import FileLink
FileLink(r'class_indices.json')

/content/class_indices.json

In [45]:
base_model = tf.keras.applications.MobileNet(weights = "imagenet",
                                             include_top = False,
                                             input_shape = input_shape)

base_model.trainable = False

In [46]:
inputs = keras.Input(shape = input_shape)

x = base_model(inputs, training = False)
x = tf.keras.layers.GlobalAveragePooling2D()(x)
x = tf.keras.layers.Dropout(0.2)(x)
x = tf.keras.layers.Dense(len(categories), 
                          activation="softmax")(x)

model = keras.Model(inputs = inputs, 
                    outputs = x, 
                    name="PlantDiseases")

In [47]:
optimizer = tf.keras.optimizers.Adam()

model.compile(optimizer = optimizer,
              loss = tf.keras.losses.CategoricalCrossentropy(from_logits = True),
              metrics=[keras.metrics.CategoricalAccuracy(), 
                       'accuracy'])

In [48]:
history = model.fit(train_data,
                    validation_data=test_data,
                    epochs=epochs,
                    steps_per_epoch=100,
                    validation_steps=50)

Epoch 1/20


/usr/local/lib/python3.7/dist-packages/tensorflow/python/util/dispatch.py:1082: UserWarning: "`categorical_crossentropy` received `from_logits=True`, but the `output` argument was produced by a sigmoid or softmax activation and thus does not represent logits. Was this intended?"
  return dispatch_target(*args, **kwargs)


100/100 [==============================] - 1255s 13s/step - loss: 0.9122 - categorical_accuracy: 0.7100 - accuracy: 0.7100 - val_loss: 0.3074 - val_categorical_accuracy: 0.9294 - val_accuracy: 0.9294
Epoch 2/20
100/100 [==============================] - 950s 10s/step - loss: 0.2790 - categorical_accuracy: 0.9225 - accuracy: 0.9225 - val_loss: 0.1536 - val_categorical_accuracy: 0.9625 - val_accuracy: 0.9625
Epoch 3/20
100/100 [==============================] - 763s 8s/step - loss: 0.2051 - categorical_accuracy: 0.9425 - accuracy: 0.9425 - val_loss: 0.1200 - val_categorical_accuracy: 0.9700 - val_accuracy: 0.9700
Epoch 4/20
100/100 [==============================] - 620s 6s/step - loss: 0.1563 - categorical_accuracy: 0.9528 - accuracy: 0.9528 - val_loss: 0.0941 - val_categorical_accuracy: 0.9756 - val_accuracy: 0.9756
Epoch 5/20
100/100 [==============================] - 495s 5s/step - loss: 0.1340 - categorical_accuracy: 0.9597 - accuracy: 0.9597 - val_loss: 0.0829 - val_categorical_acc